In [1]:
import numpy as np
import pandas as pd

In [9]:
DATA_DIR = '../../../In-situ Meas Data/Build Command Data/XYPT Commands/' # Directory of XYPT files

dt = 1E-5 # 10 microseconds

In [7]:
data = pd.read_csv(DATA_DIR + 'T500_3D_Scan_Strategies_fused_layer0002.csv', header=None, names=['X', 'Y', 'P', 'T', 'deprecated'])

In [10]:
trigger_indx = np.where(np.array(data['T']))[0]

1e-05

In [22]:
# Speed


In [28]:
# Direction? 


In [23]:
# Acceleration


In [24]:
# Energy Density


In [25]:
# Heating/Cooling


In [26]:
# Turning


In [27]:
# Residual Heat
